<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Deep Q-Learning para entrenar un jugador de Atari</center></span>

<center>Entrenamiento de una agente para jugar Pong</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Adaptado de Rowel Atienza, Advance Deep Learning with Tensorflow 2 and Keras,Pack,2020](https://www.amazon.com/-/es/Rowel-Atienza-ebook/dp/B0851D5YQQ).
1. [Sutton, R. S., & Barto, A. G. (2018).Reinforcement learning: An introductio, MIT Press, 2018](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf)
1. [Ejecutar en Colab](https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=2XelFhSJGWGX)
1. [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Envolturas (Wrappers)](#Envolturas-(Wrappers))
* [Modelo DQN](#Modelo-DQN)

* [Ecuación de Bellman en DQN](#Ecuación-de-Bellman-en-DQN)
* [Experiencia por repetición](#Experiencia-por-repetición)
* [Congelando la red objetivo](#Congelando-la-red-objetivo)
* [Algoritmo DQN](#Algoritmo-DQN)
* [Ejemplo ambiente CartPole-v1 ](#Ejemplo-ambiente-CartPole-v1-en-Gym)
* [Implementación del algoritmo DQN. CartPole](#Implementación-del-algoritmo-Q-learning.-CartPole)
* [Video luego de entrenado el agente](#Video-luego-de-entrenado-el-agente)


## <span style="color:blue">Introducción</span>

En lección entrenaremos una agente cpaza de jugar y ganar uno delos priemwros video-juegos de la historia: el juego de Atari Pong. La pri3mra versión fue introducida por Mnih et al. en un artículo de 2015 publicado por Nature como [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf). 

El problema de atacar los juegos de Atari con aprendizaje reforzado (AR) presenta varias complicaciones desde el punto de vista de los recursos que es necesario resolver. Primero es espacio de estados es finito.

Para empezar, se  tiene una grilla de una tamaño por ejemplo de 210\*160 pixels a color. Esto siginifica que a primera vista el agente puede recibir  una imagen completa del estado actual del ambiente representado en un tensor de tamaño 210\*160\*3. 

En segudo lugar, de un paso de tiempo al siguiente no hay un cambio significativo en la imagen. Esto implica que si se considera cada imagen de forma separada en el entrenamiento, no es fácil reconocer la dinámica del ambiente.

Por estas razones los científicos de AR, empezando con Minh et. al., usualmente hacen transformaciones a la interface del juego para resolver estos problemas. La plataforma OpenAP-Gym dispone de un ambiente para los juegos de Atari. Gracias a ello, los científicos utilizan las envolturas (Wrappers) para implementar tales transformaciones.

## <span style="color:blue">Envolturas (Wrappers)</span>

Las transformaciones más conocidas por os científicos del AR  son la siguientes.

1. Convertir cada vida en un episodio. Clásicamente un episodio termina cuando el juego termina (*game over*). Pero para entrenar al agente es suficiente usar como episodio cada vida.
1. Al comienzo del juego se ejeucta un salto de un número aleatorio de acciones. Esto permite evitar algunas escenas introductorias de los juegos de Atari.
1. Cada selección de la siguiente acción se hace cada $K$ pasps (frames). Usualmente $K$ es 3 o 4. Para cada frame intermedio la aación simplemente es repetida. Esta estrategia acelera significativamente el entrenamiento. Tomar cada frame individualmente, es un poco más exigente en recursos, pero la diferencia entre frames consecutivos es por lo general pequeña.
1. Se toma el máximo de cada pixel en los últimos dos frames y se usa como una observación. El problema que se resuelve en este caso, es que algunos juegos de Atari tienen un efecto de parpadeo, debido a algunas limitaciones de la plataforma de la época. Por lo general el efecto es invisible al ojo humano, pero puede confundir a la red neuronal.
1. Presionar **FIRE** al comienzo del juego. olgunos juegos incluido Pong, requieren que el usuario p1. resione **FIRE** al comienos del juego.
1. Reescalar los frame a  tamaño 84\*84 a u solo color (escala de grises).
1. Acumular varios frames seguidos (sualmente 4) para tratarlos como uso solo. Esta estrategia permite detectra mejo la dimpanica del juego.
1. Establecer las recompsensas en -1, 0, 1. El sistema de recompensas varia enlos diferentes juegos. En Pong, se entrega 1, cada vez que el oponente deja pasar la bola, -1, cada vez que el agente deja pasar la bola y 0 en el resto del juego.
1. Covertir la observaciones ala escala $[0, 1]$ para evitar problemas  a la red enel entreanamiento. Valores entre 0 y 255 son problemáticos en general.

La siguiente implementación de las envoluturas par el juego Pong es adaptada del texto de [Maxim Lapan](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67).

Las clases estan diseñadas para modificar el ambiente, implementando las transformaciones descritas.



### Clase  FireResetEnv

Implementa presionar **FIRE**, requerido en Pong.

### Clase MaxAndSkipEnv

Permite recibir varios frames,  4 por defecto, mediante al utilización de la misma acción. Se conservan únicamente los dos últimos frames en la cola (collections.deque). La nueva observación consiste en tomar el máximmo  (max pooling) por cada pixel de los dos frame conservados.

### Clase ProcessFrame84

Transforma la observación al tamaño 84\*84\*1.

### Clase ImageToPyTorch

Transforma el la observación de tipo (Ancho, alto, canal) -> (canal, ancho, alto) requirido por Pytorch.

### Clase ScaledFloatFrame

Para transformar la observación a la escala $[0,1]$ para evitar problemas de convergencia de la red.

### Clase ScaledFloatFrame

ESta es la clase que crea la pila de observaciones que finalmente es la observación que le llega al agente.

In [ ]:
import cv2
import gym
import gym.spaces
import numpy as np
import collections


class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None)
        """imula presionar FIRE para iniciar el juego."""
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        """Retorna solamente cada  `skip`-ésimo frame"""
        super(MaxAndSkipEnv, self).__init__(env)
        # conserva las más recientes obs( 2 para max pooling a lo largo de los pasos en el tiempo)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        """limpia el pasado buffer e incializa para la priemra observación desde 
        el env interno."""
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(  np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(   np.float32)
        else:
            assert False, "Resolución desconocida."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + \
              img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        new_shape = (old_shape[-1], old_shape[0], old_shape[1])
        self.observation_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=new_shape, dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(
            old_space.low.repeat(n_steps, axis=0),
            old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(
            self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


def make_env(env_name):
    env = gym.make(env_name)
    env = MaxAndSkipEnv(env)
    env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    return ScaledFloatFrame(env)


## <span style="color:blue">Modelo DQN</span>

La siguiente celda muestra la implementación de la red que se usará para el entrenamiento del agente. La diferencia con la lección de CartPole es que en esta caso, tenem osun  red convolucional de regresión. Como antes, la entrada a la red es una observación y a la salida los valores de las acciones para la siguiente acción dada la obssrvación de entrada.

Lo único por explicar en estre caso es la razón por la cual separamos la arquitectura en dos partes. Esto se debe a que a diferencia de Keras, en Pytorch no existe una capa Flatten. Entonces a la salida de la parte convolucional se  usa *view* para tener una vista del tensor de entreda como un vector plano y luegonetregarlo ala capa densa (perceptrón)-

In [ ]:
import torch
import torch.nn as nn
import numpy as np


class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)


## <span style="color:blue">Algortimo Q-Learning profundo</span>

Mnih et al. introdujeron el algoritmo Deep Q-Network o DQN, publicado en Nature en 2015 como [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf). Este algoritmo combina el algoritmo *Q-learning* con redes neuronales profundas (Deep Neural Networks). 

Como es sabido en el campo de la IA, las redes neuronales son una fantástica manera de aproximar funciones no lineales. Este algoritmo usa una red neuronal para aproximar la función *Q*, evitando así utilizar una tabla para representar la misma. 

En realidad, utiliza dos redes neuronales para estabilizar el proceso de aprendizaje. 

1. La primera, la *red neuronal principal* (main Neural Network), representada por los parámetros $\theta$, se utiliza para estimar los valores-Q del estado s y acción a actuales: $Q(s, a;\theta)$. 
2. La segunda, la *red neuronal objetivo* (target Neural Network), parametrizada por $\theta^{'}$, tendrá la misma arquitectura que la red principal pero se usará para aproximar los *valores-Q* del siguiente estado $s'$ y la siguiente acción $a'$. 

El aprendizaje ocurre en la red principal y no en la objetivo.

La red objetivo se congela (sus parámetros no se cambian) durante varias iteraciones, normalmente alrededor de 10000. Luego de transcurrido ese número de iteraciones, los parámetros de la red principal se copian a la red objetivo, transmitiendo así el aprendizaje de una a otra, haciendo que las estimaciones calculadas por la red objetivo sean más precisas.


Los datos requerido para entrenar la Q-network provienen de  la experiencia del agente: $(s_0 a_0 r_1 s_1, s_1 a_1 r_2  s_2,\ldots, s_{T-1} a_{T-1} r_T s_T)$. Cada muestra de entrenamiento es una unidad de experiencia, $s_t a_t r_{t+1} s_{t+1}$.

En el tiempo $t$ se tiene el estado $s=s_t$.  La acción $a=a_t$ es determinada usando el algoritmo Q-learning, en donde la selección de la acción es de tipo epsilon-greedy., es decir, la siguiente acción, digamos $\pi(a)$ se obtiene como:

$$
\pi(a) = \begin{cases} \text{muestra aleatoria}(a) &\text{ con probabilidad } \epsilon \\
\arg \max_a{ Q(s,a)} &\text{ en otro caso }\end{cases}
$$

## <span style="color:blue">Ecuación de Bellman en DQN</span>

La ecuación de Bellman en el caso DQN se expresa como:

$$
Q(s,a; \theta) = r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'}).
$$

Para poder entrenar una red neuronal, necesitamos una función de pérdida o coste (loss or cost function), la cual definimos como el cuadrado de la diferencia entre ambos lados de la ecuación de Bellman:

$$
L(\theta)= \mathbb{E}[  r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'})- Q(s,a; \theta)]^2.
$$


Esta será la función que minimizaremos usando el algoritmo de descenso de gradiente (gradient descent), el cuál se ejecuta automáticamente si usamos una librería de diferenciación automática con redes neuronales, como TensorFlow o Pytorch.

Desde el punto de vista del algoritmo,  se usa la $Q$-network para predecir el valor $Q$ de cada posible siguiente acción, dado el estado siguiente y escogiendo el máximo entre ellos, como henos hecho antes. 

En el caso en el cual el estado es terminal se tiene que $\max_{a^{'}}  Q(s^{'},a^{'})=0$.

### Implementación de la ecuación de Bellman en DQN: diferencia temporal

La diferencia entre la parte izquierda y la parte derecha de la ecuacipon de Bellman es denominada diferencia temporal. Si, $\Delta$ denota tal diferencia se tiene que

$$
\Delta_{s,a} = Q(s,a; \theta) - (r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'})).
$$

Esta cantidad $\Delta$ representa la diferencia entre  el Q-valor actual $Q(s,a)$ y el nuevo Q-valor que se obtiene al ejecutar la acción $a$ y alcanzar el nuevo estado $s'$.

Lo que se hace se espera es que luego de muchos pasos de experiencia acumulada, esta diferencia sea muy pequeña. En otras palabras, que el agente haya encontrado una buena aproximación de la política optimal para el ambiente. 

Lo que se busca entonces es minimizar la esperanza de $\Delta^2$, sobre espacio de estados por acciones.


Para la implementación en el algortimo se hace lo siguiente:

* La parte izquierda de la  ecuación, es decir, $Q(s,a; \theta)$ se calcula con la red principal, es decir la red que estamos entrenando.
* La parte derecha de la ecuación, es decir, $r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'})$ se calcula con la red objetivo (target).

## <span style="color:blue">Experiencia por repetición</span>


Por lo general el entrenamiento de la Q-network es inestable. Hay dos causas de la instabilidad

1. Alta correlación entre las muestras.
2. El target no es estacionario.

Para resolver el problema de la alta correlación, los datos de entrenamiento son seleccionados aleatorios de un buffer que creamos para tal fín. Este proceso se conoce como experience replay (experiencia por repetición).

## <span style="color:blue">Congelando la red objetivo</span>

EL problema de no estacionariedad de la red objetivo (target) es resuelto congelando sus pesos durante $C$ pasos de  entrenamiento. En realidad, esta es la razón de tener dos redes con idéntica arquitectura. Los parámetros de la Q-network target son copiados desde la Q-network principal cada $C$ pasos de entrenamiento.

## <span style="color:blue">Algoritmo DQN </span>

El algoritmo DQN es de tipo epsilon-greedy. 

**Primero se crean los objetos necesarios y se inicializan.**

- Inicializar la memory  replay (buffer) *D* con capacidad *N*.
- Inicializar la función de acción-valor $Q_{main}$ con pesos aleatorios $\theta$
- Inicializar la función acción-valor target $Q_{\text{target}}$ con pesos con pesos $\theta^{-}= \theta$
- Definir la rata de exploración inicial $\epsilon$ y el factor de descuento, $\gamma$.

**Ciclo de aprendizaje**

1. Para $\text{ episodio } = 1,\ldots, M$, do:
2.        Dado el estado incial s
3.        Para step = 1,...,T haga:
4.            Escoja la acción (epsilon-greddy)
$$
a \gets \begin{cases} \text{muestra aleatoria}(a) &\text{ con probabilidad } \epsilon \\
\arg \max_a{ Q(s,a)} &\text{ en otro caso }\end{cases}
$$
5. Ejecuta la acción *a* observe la recompensa *r* y el siguiente estado *s'*
6. Almacene la transición $(s,a,r,s')$ en el buffer D
7. Actualice el estado $s=s'$

**Experiencia por repetición**

8.Tome un mini-lote de muestra de experiencias de episodios $(s_j,a_j, r_{j+1}, s_{j+1})$ desde el buffer *D*.

9. $$
Q_{\text{max}} = \begin{cases} r_{j+1} &\text{ si el episodio termina en } j+1\\
r_{j+1} + \gamma \max_{a_{j+1}} Q_{\text{target}}(s_{j+1},a_{j+1};\theta ^{-}) &\text{ en otro caso }\\
\end{cases}
$$
10. Ejecuta un paso de gradiente descendiente para la función de pédida $\left(Q_{\text{max}}- Q(s_j,a_j;\theta) \right)^2$ con respecto a los parámetros

**Actualización periódica de la red objetivo(target)**

11. Cada $C$ pasos, $Q_{\text{target}}=Q_{main}$, es decir $\theta^{-} = \theta$
12. fin
13. fin


## <span style="color:blue">Implementación del algoritmo DQN. Atari-Pong</span>

En esta sección implementamos el Algortimo DQN. Usaremos el ambiente Pong ("PongNoFrameskip-v4"), como ejemplo, aunque el código es bastante genérico. Se presenta la implementación en Pytorch y se deja como ejercio la implementación en Tensorflow.

El código puede ser un poco lento en su máquina local, si no dispone de GPU's. Si eso ocurre ejecútelo en Colab.

#### Tupla nombrada Experience

Esta tupla nombrada es para organizar los datos que irán al buffer de experiencia. Sus elementos son (estado, acción, recompensa, hecho y nuevo estado).

#### Clase ExperienceBuffer

Con esta clase se crea y manipula el buffer que mantiene la experiencia adquirida por el agente a medida que explora el ambiente. El buffer es una cola de tipo *deque*. 

- El constructor *\_\_init\_\_()* crea el buffer de tamaño máximo *capacity*.
- El método *append()* agrega elementos al buffer.
- El método *sample()* extare una muestra aleatoria del buffer con tamaño *batch_size*

#### Clase Agent

Implementa al agente.

- El constructor *\_\_init\_\_()* referencia localmente una ambiente y el el buffer de experiencia. reincia el ambiente.
- El método privado *\_reset()* reinicia el ambiente y la recompensa total de un episodio.
- El métod *play_step()* ejecuta un paso de experiencia: selecionauna acción usando la técnica epsilon-greddy, le informa al ambiente, recibe la retroalimentación del ambienete y sube la experiencia la buffer. 

#### Función calc_loss()

Calcula la función de pérdida MSE con base en la diferencia temporal explicada arriba.

In [7]:
import numpy as np
import collections 

import torch
import torch.nn as nn


# namedtuple para cosntruir la tupla (estado, acción, recompensa, hecho y nuevo estado)
Experience = collections.namedtuple(
    'Experience', field_names=['state', 'action', 'reward',
                               'done', 'new_state'])

# Clase para manipular  el buffer de experiencia
class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = \
            zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), \
               np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), \
               np.array(next_states)


class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward,
                         is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

# función de pérdida
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(np.array(
        states, copy=False)).to(device)
    next_states_v = torch.tensor(np.array(
        next_states, copy=False)).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)

    state_action_values = net(states_v).gather(
        1, actions_v.unsqueeze(-1)).squeeze(-1)
    with torch.no_grad():
        next_state_values = tgt_net(next_states_v).max(1)[0]
        next_state_values[done_mask] = 0.0
        next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + \
                                   rewards_v
    return nn.MSELoss()(state_action_values,
                        expected_state_action_values)


###  Entrenamiento

In [ ]:
from lib import wrappers
from lib import dqn_model

import time
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter


DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
MEAN_REWARD_BOUND = 19

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

EPSILON_DECAY_LAST_FRAME = 150000
EPSILON_START = 1.0
EPSILON_FINAL = 0.0


# define el dispositovo que usará
device = torch.device("cuda" if args.cuda else "cpu")

# define el ambiente
env = wrappers.make_env(args.env)

# Crea las dos redes
net = dqn_model.DQN(env.observation_space.shape,
                    env.action_space.n).to(device)
tgt_net = dqn_model.DQN(env.observation_space.shape,
                        env.action_space.n).to(device)
# inicia el SummaryWriter
writer = SummaryWriter(comment="-" + args.env)
print(net)

# Crea el buffer de experiencia
buffer = ExperienceBuffer(REPLAY_SIZE)

# Crea el agente
agent = Agent(env, buffer)

# define le epsilon inicial para la política epsilon-voráz
epsilon = EPSILON_START

# instancia el optimizador Adam
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

# Inicializa objetos de trabajo
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

# ciclo de entrenamiento
while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START -
                  frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, epsilon, device=device)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        print("%d: done %d games, reward %.3f, "
              "eps %.2f, speed %.2f f/s" % (
            frame_idx, len(total_rewards), m_reward, epsilon,
            speed
        ))
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or best_m_reward < m_reward:
            torch.save(net.state_dict(), args.env +
                       "-best_%.0f.dat" % m_reward)
            if best_m_reward is not None:
                print("Best reward updated %.3f -> %.3f" % (
                    best_m_reward, m_reward))
            best_m_reward = m_reward
        if m_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device=device)
    loss_t.backward()
    optimizer.step()
writer.close()
